In [1]:
# Importaciones necesarias al inicio
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64
import pandas as pd

# --- DataFrame para guardar las ideas de diseño ---
# Se inicializa una única vez al inicio del script
ideas_diseño_df = pd.DataFrame(columns=[
    "espacio", "estilo", "colores", "iluminacion", "elementos",
    "prompt_textual", "propuesta_ambientacion", "prompt_imagen_generado",
    "tipo_generacion" # Puedes usar esto para indicar si la generación fue "IA" o "Manual"
])

# --- Función para guardar en el DataFrame ---
def guardar_idea_diseño(espacio, estilo, colores, iluminacion, elementos,
                        prompt_textual, propuesta_ambientacion, prompt_imagen_generado,
                        tipo_generacion="IA"): # Valor por defecto "IA"
    global ideas_diseño_df # Accedemos al DataFrame global
    nueva_fila = {
        "espacio": espacio,
        "estilo": estilo,
        "colores": colores,
        "iluminacion": iluminacion,
        "elementos": elementos,
        "prompt_textual": prompt_textual,
        "propuesta_ambientacion": propuesta_ambientacion,
        "prompt_imagen_generado": prompt_imagen_generado,
        "tipo_generacion": tipo_generacion
    }
    ideas_diseño_df = pd.concat([ideas_diseño_df, pd.DataFrame([nueva_fila])], ignore_index=True)
    print("✅ Idea de diseño guardada correctamente en el DataFrame.")

In [2]:
# --- Paso 1: Inputs del usuario ---
espacio = "living"
estilo = "boho chic"
colores = "off white, marrones y fibras naturales"
iluminacion = "luz natural"
elementos = "plantas, muebles de madera clara, alfombra tejida"

In [3]:
# --- Paso 2: Generación del prompt para la descripción textual ---
prompt_textual = f"""
Sugiéreme una propuesta de ambientación para un {espacio} de tamaño pequeño con estilo {estilo}.
Debe incluir colores como {colores}, con {iluminacion} y elementos como {elementos}.
"""
print("🔹 Prompt para generar descripción de ambientación:")
print(prompt_textual)


🔹 Prompt para generar descripción de ambientación:

Sugiéreme una propuesta de ambientación para un living de tamaño pequeño con estilo boho chic.
Debe incluir colores como off white, marrones y fibras naturales, con luz natural y elementos como plantas, muebles de madera clara, alfombra tejida.



In [4]:
# --- Paso 3: Conexión a Gemini (Google Generative AI) y generación de texto ---

# Clave de API
# Por seguridad, es recomendable NO dejar la API KEY directamente en el código
# Considera usar variables de entorno (ej. os.environ.get("GOOGLE_API_KEY"))
API_KEY = "TU APY KEY"

# 1. Crear un cliente con la clave de API
client = genai.Client(api_key=API_KEY)

# 2. Generar la descripción textual de la ambientación
# Aquí 'text_response_gemini' contiene la respuesta completa, incluyendo el texto
text_response_gemini = client.models.generate_content(
    model="gemini-1.5-flash",
    contents=prompt_textual
)

# Guardamos el texto generado para usarlo después
propuesta_ambientacion_completa = "\n--- Propuesta de Ambientación ---\n" + text_response_gemini.text
print(propuesta_ambientacion_completa)


--- Propuesta de Ambientación ---
## Ambientación Boho Chic para Living Pequeño: "Luz y Armonía"

Esta propuesta busca maximizar la sensación de espacio y luz en un living pequeño, manteniendo la esencia boho chic a través de la combinación de texturas, colores y elementos naturales.

**Paleta de Colores:**  Base de off-white en paredes para ampliar visualmente el espacio.  Marrón en tonos cálidos (chocolate, miel, café con leche) en los muebles de madera y algunos textiles,  contrapuestos con el blanco roto y fibras naturales.

**Iluminación:**  La luz natural es clave.  Mantén las ventanas despejadas, quizás con cortinas de lino blanco o crudo semitransparentes que filtren la luz sin oscurecer el espacio.  Complementa con iluminación cálida y tenue:

* **Lámpara de techo:**  Una pieza de fibras naturales (ratán, mimbre) de tamaño moderado para no sobrecargar el espacio.
* **Lámparas de mesa:**  Dos lámparas pequeñas de madera clara con pantallas de tela en tonos beige o marrón claro

In [5]:
# --- Paso 4: Generar un prompt optimizado para la imagen ---
# Pedimos a Gemini que extraiga las palabras clave visuales del texto extenso

prompt_para_generar_imagen_optimizada = f"""
Basado en la siguiente propuesta de Decoración, genera una descripción concisa y altamente visual (máximo 150 palabras) ideal para un modelo de generación de imágenes. Enfócate en los elementos clave, colores, estilo y ambiente.

Propuesta de ambientación:
{propuesta_ambientacion_completa}

Descripción visual para imagen:
"""

image_prompt_response_gemini = client.models.generate_content(
    model="gemini-1.5-flash", # Usamos el mismo modelo de texto para esto
    contents=prompt_para_generar_imagen_optimizada
)

#  prompt de imagen sea solo texto y conciso
image_text_for_generation = image_prompt_response_gemini.text.strip()
print(f"\n--- Prompt optimizado para la imagen ---\n{image_text_for_generation}")


--- Prompt optimizado para la imagen ---
Living pequeño estilo boho chic luminoso. Paredes off-white, sofá beige claro, mesa de centro madera clara con bandeja de mimbre.  Alfombra tejida beige y marrón claro.  Iluminación cálida: lámpara de techo de ratán, lámparas de mesa madera clara con pantallas beige.  Cojines en tonos tierra con detalles étnicos. Plantas en macetas de cerámica y fibra natural.  Detalles en madera, mimbre y metal oscuro. Ambiente acogedor y minimalista, con  textura y armonía.


In [6]:
# ---- Paso 5 Guarda la información en el DataFrame AQUÍ ---
guardar_idea_diseño(
    espacio=espacio,
    estilo=estilo,
    colores=colores,
    iluminacion=iluminacion,
    elementos=elementos,
    prompt_textual=prompt_textual,
    propuesta_ambientacion=propuesta_ambientacion_completa,
    prompt_imagen_generado=image_text_for_generation,
    tipo_generacion="IA" # Asumimos que esta generación es por IA
)
# --- Opcional: Imprime el DataFrame para verificar ---
print("\n--- Contenido actual del DataFrame de ideas de diseño ---")
print(ideas_diseño_df)
print("-------------------------------------------------------")


✅ Idea de diseño guardada correctamente en el DataFrame.

--- Contenido actual del DataFrame de ideas de diseño ---
  espacio     estilo                                 colores  iluminacion  \
0  living  boho chic  off white, marrones y fibras naturales  luz natural   

                                           elementos  \
0  plantas, muebles de madera clara, alfombra tejida   

                                      prompt_textual  \
0  \nSugiéreme una propuesta de ambientación para...   

                              propuesta_ambientacion  \
0  \n--- Propuesta de Ambientación ---\n## Ambien...   

                              prompt_imagen_generado tipo_generacion  
0  Living pequeño estilo boho chic luminoso. Pare...              IA  
-------------------------------------------------------


In [9]:
# --- Paso 6: Generar la imagen basada en el prompt optimizado ---

# Usamos directamente la variable 'image_text_for_generation' que contiene el prompt visual.
contents = (image_text_for_generation)

response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation", # Asegúrate de que este modelo esté disponible para tu API Key y región
    contents=contents,
    config=types.GenerateContentConfig(
        response_modalities=['TEXT', 'IMAGE']
    )
)

for part in response.candidates[0].content.parts:
    if part.text is not None:
        print(part.text)
    elif part.inline_data is not None:
        image = Image.open(BytesIO((part.inline_data.data)))
        image.save('gemini-native-image.png')
        image.show()

Se generará una imagen de un salón pequeño de estilo boho chic muy luminoso. Las paredes serán de un blanco roto, contrastando suavemente con un sofá de un beige muy claro. En el centro, una mesa de centro de madera clara sostendrá una bandeja de mimbre. Bajo el sofá, una alfombra tejida en tonos beige y marrón claro aportará calidez. La iluminación será cálida, con una lámpara de techo de ratán como punto focal y lámparas de mesa con bases de madera clara y pantallas beige estratégicamente ubicadas. Varios cojines en tonos tierra, adornados con detalles étnicos, decorarán el sofá. Se incluirán plantas en macetas de cerámica y fibra natural, añadiendo un toque de frescura. Detalles decorativos en madera, mimbre y sutiles acentos de metal oscuro completarán la escena, creando un ambiente acogedor, minimalista, rico en texturas y armonía visual.

